In [1]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from config import Config

c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


RuntimeError: Failed to import transformers.models.auto because of the following error (look up to see its traceback):
No module named 'transformers.models'

In [ ]:
class UniMSRAG:
    """
    Đây là phần cốt lõi thực hiện logic UniMS-RAG  gồm 3 bước:
    - Knowledge Source Selection (Planner): Quyết định có cần tìm kiếm thông tin không.
    - Knowledge Retrieval (Retriever): Tìm kiếm vector.
    - Response Generation (Reader): Sinh câu trả lời.
    """
    def __init__(self, llm, qdrant_client):
        self.llm = llm
        self.client = qdrant_client
        self.encoder = SentenceTransformer(Config.EMBEDDING_MODEL)
    
    def planner(self, user_query):
        """
        Step 1: Knowledge Source Selection [cite: 38, 139]
        Quyết định xem câu hỏi có cần tra cứu menu/thông tin nhà hàng không.
        """
        prompt = (
            f"Câu hỏi: \"{user_query}\"\n"
            "Hãy xác định xem câu hỏi này có cần tra cấu thông tin về menu (giá, món ăn) "
            "hoặc thông tin nhà hàng (địa chỉ, giờ mở cửa) không?\n"
            "Nếu CÓ, hãy trả lời: [SEARCH]\n"
            "Nếu KHÔNG (ví dụ: chào hỏi, cảm ơn), hãy trả lời: [NO_SEARCH]"
        )
        response = self.llm.generate(prompt, max_new_tokens=10)
        print("[1] " + response)
        if "SEARCH" in response:
            return "SEARCH"
        return "NO_SEARCH"
    
    def retriever(self, user_query, top_k=3):
        """
        Step 2: Knowledge Retrieval [cite: 40, 141]
        Tìm kiếm các đoạn văn bản liên quan trong Qdrant.
        """
        query_vector = self.encoder.encode(user_query).tolist()
        hits = self.client.search(
            collection_name=Config.COLLECTION_NAME,
            query_vector=query_vector,
            limit=top_k
        )
        results = [hit.payload['text'] for hit in hits]
        print("[2] " + results)
        return results
    
    def reader(self, user_query, retrieved_contexts):
        """
        Step 3: Response Generation [cite: 41, 143]
        Sinh câu trả lời dựa trên ngữ cảnh đã tìm được.
        """
        context_str = "\n".join([f" - {c}" for c in retrieved_contexts])
        
        print("[3] " + context_str)
        prompt = (
            "Dưới đây là thông tin từ cơ sở dữ liệu của nhà hàng Hòa Viên (Hoa Vien Restaurant):\n"
            f"{context_str}\n\n"
            "Hãy đóng vai là một nhân viên phục vụ thân thiện, chuyên nghiệp của nhà hàng Hòa Viên. "
            "Dựa vào thông tin trên, hãy trả lời câu hỏi của khách hàng.\n"
            f"Khách hàng: {user_query}\n"
            "Trả lời:"
        )
        return self.llm.generate(prompt)

    def process(self, user_query):
        # 1. Plan
        action = self.planner(user_query)
        
        # 2. Retrieve
        contexts = []
        if action == "[SEARCH]":
            contexts = self.retriever(user_query)
        
        # 3. Generate
        if contexts:
            return self.reader(user_query, contexts)
        else:
            # Nếu không cần search, trả lời trực tiếp (chitchat)
            prompt = (
                "Bạn là nhân viên nhà hàng Hòa Viên. "
                f"Khách hàng nói: \"{user_query}\". "
                "Hãy phản hồi một cách lịch sự, ngắn gọn."
            )
            return self.llm.generate(prompt)

In [10]:

# B5: Init Chatbot
bot = UniMSRAG(llm, client)
print("\n✅ System Ready!")

# --- TEST CASE ---
queries = [
    "Giá lẩu cua đồng là bao nhiêu?", # Case 2: Cần search Menu
    "Giá Há cảo bách hoa là bao nhiêu?", # Case 2: Cần search Menu
    "Nhà hàng mở cửa lúc mấy giờ?",
    "Tôi muốn ăn món gì đó cay cay",
    "Tôi muốn ăn gì đó cay cay",
    "Có những món nào trong menu?",
    "Tôi đã đặt những món gì?",
    "Ơ đây có bán món gì 50000 không"
    "Xin chào, bạn khỏe không?",
]

print("\n" + "="*50)
print("BẮT ĐẦU HỎI ĐÁP")
print("="*50)

for q in queries:
    print(f"\n👤 User: {q}")
    try:
        response = bot.process(q)
        print(f"🤖 Bot: {response}")
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`



✅ System Ready!

BẮT ĐẦU HỎI ĐÁP

👤 User: Xin chào, bạn khỏe không?
[1] [NO_SEARCH]
🤖 Bot: Xin chào! Tôi tốt lắm, cảm ơn đã hỏi. Bạn cần tôi giúp gì hôm nay không?

👤 User: Giá lẩu cua đồng là bao nhiêu?
[1] [SEARCH]
🤖 Bot: Chào quý khách, giá lẩu cua đồng của chúng tôi hiện tại là 150.000đ cho một suất. Mong quý khách sẽ ủng hộ nhà hàng của chúng tôi.

👤 User: Nhà hàng mở cửa lúc mấy giờ?
[1] [NO_SEARCH]
🤖 Bot: Chào quý khách, nhà hàng của chúng tôi bắt đầu phục vụ từ 11:00 đến 22:00 mỗi ngày. Xin cảm ơn!

👤 User: Tôi muốn ăn gì đó cay cay
[1] [NO_SEARCH]
🤖 Bot: Chắc chắn, chúng tôi có nhiều món ngon với vị cay cay như các món gỏi, nem rán hay các loại mì đặc biệt. Bạn muốn thử món nào ngay bây giờ không?

👤 User: Ơ đây có bán món gì 50000 không
[1] [SEARCH]
🤖 Bot: Xin lỗi đã vướng phải sự hiểu lầm, chúng tôi hiện đang phục vụ các món với mức giá từ 30,000 đến 60,000 VND. Mong quý khách thông cảm.


In [ ]:
from ingest import DataIngestor
from llm_wrapper import LLMWrapper
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

print("="*50)
print("STARTING FULL SYSTEM TEST")
print("="*50)
# B2: Khởi tạo Qdrant (Memory Mode)

# Vì config để ":memory:", tắt code đi là mất dữ liệu -> Phải ingest lại mỗi lần chạy
print("\n[1/4] Connecting to Qdrant...")
client = QdrantClient(location=":memory:")

# B3: Ingest Data (Nạp dữ liệu vào RAM)
print("\n[2/4] Ingesting Data...")
ingestor = DataIngestor(client)
ingestor.ingest()
    
# B4: Load LLM
print("\n[3/4] Loading LLM (Wait for model loading)...")
llm = LLMWrapper() # Class này của bạn sẽ tự load Qwen và tokenizer


c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


STARTING FULL SYSTEM TEST

[1/4] Connecting to Qdrant...

[2/4] Ingesting Data...


c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--AITeamVN--Vietnamese_Embedding. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling ba

--- Bắt đầu nạp dữ liệu vào Vector DB ---


c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


--- Đã nạp 30 documents vào Qdrant ---

[3/4] Loading LLM (Wait for model loading)...
Loading LLM: Qwen/Qwen2.5-3B-Instruct...


c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--Qwen--Qwen2.5-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.56s/it]
Some parameters are on 

LLM Loaded successfully.


NameError: name 'UniMSRAG' is not defined